In [6]:
##1 branch and college according to Rank
import pandas as pd
import numpy as np

# Load datasets
file = pd.read_csv('combined2.csv')
seats_file = pd.read_csv('Seat_State_Matrix.csv')
seats_file.dropna(subset='Total (includes Female Supernumerary)', inplace=True)

# Filter IIT and non-IIT data for round 6 and year 2023
file1 = file.loc[
    (file['Round'] == 6) & 
    (file['Year'] == 2023) & 
    (file['Institute'].str.contains('Indian Institute  of Technology', case=False, na=False))
]

file2 = file.loc[
    (file['Round'] == 6) & 
    (file['Year'] == 2023) & 
    (~(file['Institute'].str.contains('Indian Institute  of Technology', case=False, na=False)))
]

# Get user inputs
adv_rank = int(input("Enter Your Advanced Rank:"))
mains_rank = int(input("Enter Your Mains Rank:"))
seatty = input("Enter Your seat type(OPEN,EWS,OBC-NCL,SC,ST,EWS (PwD),OBC-NCL (PwD),OPEN (PwD), SC (PwD), ST(PwD)):")
gender = input("Enter Your Gender(Male,Female):")
state = input("Enter Your State:")
state = state.upper()

# Filter state quota from seats_file
state_quota = seats_file.loc[seats_file['State/All India Seats'] == state]
home_state_institute = state_quota['Institute Name']

# Define gender list based on input
gender_list = ['Female-only (including Supernumerary)'] if gender.lower() == 'female' else ['Gender-Neutral']

# Convert ranks to numeric, coercing errors to NaN, and drop rows with NaN values in relevant columns
for df in [file1, file2]:
    df['Opening Rank'] = pd.to_numeric(df['Opening Rank'], errors='coerce')
    df['Closing Rank'] = pd.to_numeric(df['Closing Rank'], errors='coerce')
    df.dropna(subset=['Closing Rank', 'Institute'], inplace=True)

# Filter choices for IITs
choices_iit = file1.loc[
    (adv_rank >= file1['Opening Rank']) & 
    (adv_rank <= file1['Closing Rank']) & 
    (seatty.upper() == file1['Seat Type']) & 
    (file1['Gender'].isin(gender_list))
]

# Filter choices for other institutes
choices_other = file2.loc[
    (mains_rank >= file2['Opening Rank']) & 
    (mains_rank <= file2['Closing Rank']) & 
    (seatty.upper() == file2['Seat Type']) & 
    (file2['Gender'].isin(gender_list)) & 
    (
        (file2['Quota'] == 'OS') | 
        (file2['Quota'] == 'AI') )
]

choices_home = file2.loc[(mains_rank >= file2['Opening Rank']) & 
    (mains_rank <= file2['Closing Rank']) & 
    (seatty.upper() == file2['Seat Type']) & 
    (file2['Gender'].isin(gender_list)) &
    (file2['Quota']== 'HS') &
    (file2['Institute'].values[0] == home_state_institute)
]

# Concatenate and sort the choices
choices_other = pd.concat([choices_home,choices_other])
choices = pd.concat([choices_iit, choices_other]).sort_values(by='Opening Rank')

# Print the filtered choices without index
print(choices[['Institute', 'Academic Program Name']].to_string(index=False))

C:\Users\91934\AppData\Local\Temp\ipykernel_22672\2787185437.py:6: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv('combined2.csv')


                                                                                                                    Institute                                                                                                                                Academic Program Name
                                                                     Indian Institute of Information Technology(IIIT) Dharwad                                                                                   Computer Science and Engineering (4 Years, Bachelor of Technology)
                                                                          Indian Institute of Information Technology Guwahati                                                                          Electronics and Communication Engineering (4 Years, Bachelor of Technology)
                                                                                Indian Institute  of Technology (ISM) Dhanbad                                                  

C:\Users\91934\AppData\Local\Temp\ipykernel_22672\2787185437.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Opening Rank'] = pd.to_numeric(df['Opening Rank'], errors='coerce')
C:\Users\91934\AppData\Local\Temp\ipykernel_22672\2787185437.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Closing Rank'] = pd.to_numeric(df['Closing Rank'], errors='coerce')
C:\Users\91934\AppData\Local\Temp\ipykernel_22672\2787185437.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [29]:
##2 Popular Branches
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# Load the dataset
df = pd.read_csv('combined2.csv')

# Ensure the relevant columns are present and convert ranks to numeric
df['Opening Rank'] = pd.to_numeric(df['Opening Rank'], errors='coerce')
df['Closing Rank'] = pd.to_numeric(df['Closing Rank'], errors='coerce')

# Drop rows with NaN values in 'Opening Rank' or 'Closing Rank'
df.dropna(subset=['Opening Rank', 'Closing Rank'], inplace=True)

# Calculate the rank difference and average rank
df['Rank Difference'] = df['Closing Rank'] - df['Opening Rank']
df['Average Rank'] = (df['Closing Rank'] + df['Opening Rank']) / 2

# Filter the dataset based on specific conditions
df_filtered = df.loc[(df['Gender'] == 'Gender-Neutral') & (df['Quota'] == 'AI') & 
                     (df['Seat Type'] == 'OPEN') & (df['Round'] == 6) & 
                     (df['Year'] == 2023) &
                     (~df['Institute'].str.contains('Architecture', na=False)) & 
                     (~df['Academic Program Name'].str.contains('Architecture', na=False))]

# Sort by Year (descending), then by Average Rank and Rank Difference (ascending)
sorted_by_popularity = df_filtered.sort_values(by=['Average Rank', 'Rank Difference'])

# Drop duplicates based on 'Academic Program Name' to get unique branches
df_unique_branches = sorted_by_popularity.drop_duplicates(subset='Academic Program Name', keep='first')

# Select the top 10 branches based on the sorted order
top_10 = df_unique_branches.head(10)

# Display the top 10 most popular branches
print("Top 10 Most Popular Branches Based on Rank Difference:")
print(top_10['Academic Program Name'])

fig = px.bar(
    top_10,
    x='Average Rank',
    y='Academic Program Name',
    orientation='h',
    title='Average Rank of Top 10 Different Branches',
    labels={'Average Rank': 'Average Rank', 'Academic Program Name': 'Academic Program Name'},
    color='Average Rank',
    color_continuous_scale=px.colors.sequential.Blues
)
fig.update_layout(xaxis_title='Average Rank', yaxis_title='Academic Program Name', yaxis={'categoryorder':'total ascending'})
fig.show()

# If the data includes multiple years, perform trend analysis
top_institutes = df.loc[df['Year'] == 2023].groupby('Institute')['Average Rank'].mean().nsmallest(10).index.tolist()
df_filtered_all_years = df.loc[(df['Gender'] == 'Gender-Neutral') & (df['Quota'] == 'AI') & 
                               (df['Seat Type'] == 'OPEN') & (df['Round'] == 6) & 
                               (df['Institute'].isin(top_institutes)) & 
                               (df['Year'] >= 2016) & (df['Year'] <= 2023) &
                               (~df['Institute'].str.contains('Architecture', na=False)) & 
                               (~df['Academic Program Name'].str.contains('Architecture', na=False))]

if 'Year' in df.columns:
    trend_data = df_filtered_all_years[df_filtered_all_years['Academic Program Name'].isin(top_10['Academic Program Name'])]
    
    # Aggregate data to get average rank per year for each branch
    trend_data = trend_data.groupby(['Year', 'Academic Program Name']).agg({
        'Average Rank': 'mean'
    }).reset_index()

    # Create the figure for trend analysis using Plotly
    fig = go.Figure()

    for branch in top_10['Academic Program Name']:
        branch_data = trend_data[trend_data['Academic Program Name'] == branch]
        fig.add_trace(go.Scatter(
            x=branch_data['Year'],
            y=branch_data['Average Rank'],
            mode='lines+markers',
            name=f'{branch} - Average Rank',
            line=dict(width=2),  # Default line width
            opacity=0.8 # Default opacity
        ))

    fig.update_layout(
        title='Trend of Average Ranks Over Years for Top Branches',
        xaxis_title='Year',
        yaxis_title='Average Rank',
        yaxis_autorange='reversed',
        legend_title='Branches',
        hovermode='closest'
    )

    # Add custom hover effect to highlight only the hovered trace
    fig.update_layout(
        updatemenus=[{
            "buttons": [
                {
                    "args": ["mode", "lines+markers"],
                    "label": "Show All",
                    "method": "restyle"
                }
            ],
            "direction": "down",
            "showactive": True
        }]
    )

    fig.update_layout(
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        )
    )

    fig.show()

# JavaScript for hover functionality
from IPython.display import display, HTML

hover_js = """
<script>
    var myPlot = document.getElementsByClassName('plotly-graph-div')[0];

    myPlot.on('plotly_hover', function(data){
        var index = data.points[0].curveNumber;
        Plotly.restyle(myPlot, 'opacity', Array.from({length: data.points.length}, (_, i) => i === index ? 1 : 0.2));
    });

    myPlot.on('plotly_unhover', function(data){
        Plotly.restyle(myPlot, 'opacity', 0.2);
    });
</script>
"""

display(HTML(hover_js))


C:\Users\91934\AppData\Local\Temp\ipykernel_30452\1057400926.py:8: DtypeWarning:

Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.



Top 10 Most Popular Branches Based on Rank Difference:
364986    Computer Science and Engineering (4 Years, Bac...
365266    Computer Science and Engineering (5 Years, Bac...
365346    Mathematics and Computing (4 Years, Bachelor o...
365014    Electrical Engineering (4 Years, Bachelor of T...
365359    Mathematics and Computing (5 Years, Bachelor a...
366757    Data Science and Artificial Intelligence (4 Ye...
365291    Electrical Engineering (Power and Automation) ...
365864    Artificial Intelligence (4 Years, Bachelor of ...
365027    Electrical Engineering (5 Years, Bachelor and ...
366229    Mathematics and Scientific Computing (4 Years,...
Name: Academic Program Name, dtype: object


In [31]:
##3 college-wise branch popularity
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv('combined2.csv')

# Ensure the relevant columns are present and convert ranks to numeric
df['Opening Rank'] = pd.to_numeric(df['Opening Rank'], errors='coerce')
df['Closing Rank'] = pd.to_numeric(df['Closing Rank'], errors='coerce')
df.dropna(subset = 'Closing Rank',inplace=True)

combined_2023 = df.loc[df.Year==2023]

college_name = input("Enter the college name")
seatty = input("Enter Your seat type(OPEN,EWS,OBC-NCL,SC,ST,EWS (PwD),OBC-NCL (PwD),OPEN (PwD), SC (PwD), ST(PwD)):")
gender = input("Enter Your Gender(Male,Female):")
quota = input('Enter Your Quota(AI,OS,HS):')

seatty = seatty.upper()
quota = quota.upper()
gender = gender.lower()
gender_list = ['Female-only (including Supernumerary)'] if gender== 'female' else ['Gender-Neutral']

college_data = combined_2023[(combined_2023.Institute == college_name) & (combined_2023['Seat Type'] == seatty) & (combined_2023['Gender'].isin(gender_list)) &  (combined_2023['Quota'] == quota)]
college_data['Average Rank'] = (college_data['Opening Rank']+college_data['Closing Rank'])/2

college_data = college_data[['Academic Program Name', 'Average Rank']]
sorted_college_data = college_data.sort_values(by='Average Rank')
sorted_college_data.head(10).reset_index(drop=True)

fig = px.bar(
    top_10,
    x='Average Rank',
    y='Academic Program Name',
    orientation='h',
    title=f'Average Rank of Top 10 Different Branches in {college_name}',
    labels={'Average Rank': 'Average Rank', 'Academic Program Name': 'Academic Program Name'},
    color='Average Rank',
    color_continuous_scale=px.colors.sequential.Blues
)
fig.update_layout(xaxis_title='Average Rank', yaxis_title='Academic Program Name', yaxis={'categoryorder':'total ascending'})
fig.show()


C:\Users\91934\AppData\Local\Temp\ipykernel_30452\3547850700.py:6: DtypeWarning:

Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\91934\AppData\Local\Temp\ipykernel_30452\3547850700.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [35]:
#branch-wise college popularity
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv('combined2.csv')

# Ensure the relevant columns are present and convert ranks to numeric
df['Opening Rank'] = pd.to_numeric(df['Opening Rank'], errors='coerce')
df['Closing Rank'] = pd.to_numeric(df['Closing Rank'], errors='coerce')
df.dropna(subset=['Closing Rank'], inplace=True)

# Filter the dataset for the year 2023
df = df[df['Year'] == 2023]
college_data = df.loc[(df['Gender'] == 'Gender-Neutral') & 
                      (df['Academic Program Name'].str.contains(branch, case=False)) &
                     ((df['Quota'] == 'AI') | (df['Quota'] == 'OS') )& 
                     (df['Seat Type'] == 'OPEN') & 
                     (df['Round'] == 6) & 
                     (~df['Institute'].str.contains('Architecture', na=False)) & 
                     (~df['Academic Program Name'].str.contains('Architecture', na=False))]

# Calculate the average rank
college_data['Average Rank'] = (college_data['Opening Rank'] + college_data['Closing Rank']) / 2

# Further filter based on rank input
rank_adv = int(input('Enter your Advanced Rank: '))
rank_mains = int(input('Enter your Mains Rank: '))

# Filter for IITs and other institutes separately and then combine
iit_data = college_data[(college_data['Opening Rank'] >= rank_adv) & (college_data['Institute'].str.contains('Indian Institute  of Technology'))]
non_iit_data = college_data[(college_data['Opening Rank'] >= rank_mains) & (~college_data['Institute'].str.contains('Indian Institute  of Technology'))]

combined_2023 = pd.concat([iit_data, non_iit_data])

# Select and sort the top 10 colleges
top_10 = combined_2023.sort_values(by='Average Rank').head(10)

# Concatenate Institute and Academic Program Name for y-axis
top_10['Institute-Academic Program Name'] = top_10['Institute'] + ' - ' + top_10['Academic Program Name']

# Create the Plotly figure
fig = px.bar(
    top_10,
    x='Average Rank',
    y='Institute-Academic Program Name',
    orientation='h',
    title=f'Average Rank of Top 10 Different Colleges offering {branch}',
    labels={'Average Rank': 'Average Rank', 'Institute-Academic Program Name': 'Institute-Academic Program Name'},
    color='Average Rank',
    color_continuous_scale=px.colors.sequential.Blues
)

fig.update_layout(xaxis_title='Average Rank', yaxis_title='Institute-Academic Program Name', yaxis={'categoryorder':'total ascending'})

fig.show()


C:\Users\91934\AppData\Local\Temp\ipykernel_30452\1135650399.py:6: DtypeWarning:

Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\91934\AppData\Local\Temp\ipykernel_30452\1135650399.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [34]:
##5 Branch and College Trend over Years
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv('combined2.csv')
college_name = input("Enter the college name: ")
branch = input("Enter the branch name: ")
seatty = input("Enter Your seat type(OPEN,EWS,OBC-NCL,SC,ST,EWS (PwD),OBC-NCL (PwD),OPEN (PwD), SC (PwD), ST(PwD)):")
gender = input("Enter Your Gender(Male,Female):")
quota = input('Enter Your Quota(AI,OS,HS):')

df['Closing Rank'] = pd.to_numeric(df['Closing Rank'], errors='coerce')
df['Opening Rank'] = pd.to_numeric(df['Opening Rank'], errors='coerce')

df = df.dropna(subset=['Closing Rank', 'Opening Rank'])

seatty = seatty.upper()
quota = quota.upper()
gender = gender.lower()
gender_list = ['Female-only (including Supernumerary)'] if gender== 'female' else ['Gender-Neutral']
df['Average Rank'] = (df['Closing Rank'] +df['Opening Rank'])/2

college_data = df[(df.Institute == college_name) & (df['Seat Type'] == seatty) & (df['Gender'].isin(gender_list)) &  (df['Quota'] == quota) & (df['Round']==6) & (df['Academic Program Name'].str.contains(branch, case=False))]

# Sort the data by year
college_data = college_data.sort_values(by='Year')
fig = go.Figure()

# Add a trace for the line graph
fig.add_trace(go.Scatter(
    x=college_data['Year'],
    y=college_data['Average Rank'],
    mode='lines+markers',  # Show both lines and markers
    marker=dict(size=8),  # Marker size
    line=dict(color='blue'),  # Line color
    name=f'{branch} - {college_name}'  # Name for legend
))

# Update layout
fig.update_layout(
    title=f"Average Rank of {branch} in {college_name}",
    xaxis_title='Years',
    yaxis_title='Average Rank',
    xaxis=dict(tickmode='linear', tick0=2018, dtick=1),  # Set ticks for years
)

fig.show()


C:\Users\91934\AppData\Local\Temp\ipykernel_30452\1010763863.py:5: DtypeWarning:

Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.



In [47]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import display, HTML

# Load the dataset
df = pd.read_csv('combined2.csv')
df['Institute-Academic Program Name'] = df['Institute'] + ' - ' + df['Academic Program Name']

# Ensure the relevant columns are present and convert ranks to numeric
df['Opening Rank'] = pd.to_numeric(df['Opening Rank'], errors='coerce')
df['Closing Rank'] = pd.to_numeric(df['Closing Rank'], errors='coerce')

# Drop rows with NaN values in 'Opening Rank' or 'Closing Rank'
df.dropna(subset=['Opening Rank', 'Closing Rank'], inplace=True)

# Calculate the average rank
df['Average Rank'] = (df['Closing Rank'] + df['Opening Rank']) / 2

# Input parameters for filtering data
rank_adv = int(input('Enter Your Advanced Rank: '))
rank_mains = int(input('Enter Your Mains Rank: '))

# Filter data based on rank inputs
iit_data = df[(df['Opening Rank'] >= rank_adv) & df['Institute'].str.contains('Indian Institute  of Technology')]
non_iit_data = df[(df['Opening Rank'] >= rank_mains) & (~df['Institute'].str.contains('Indian Institute  of Technology'))]

df_filtered = pd.concat([iit_data, non_iit_data])

# Filter the dataset based on specific conditions
df_filtered = df_filtered.loc[(df_filtered['Gender'] == 'Gender-Neutral') & 
                              ((df_filtered['Quota'] == 'AI') | (df_filtered['Quota'] == 'OS')) & 
                              (df_filtered['Seat Type'] == 'OPEN') & 
                              (df_filtered['Round'] == 6) & 
                              (~df_filtered['Institute'].str.contains('Architecture', na=False)) & 
                              (~df_filtered['Academic Program Name'].str.contains('Architecture', na=False))]

# Identify unique branches from the filtered data
df_unique_branches = df_filtered.drop_duplicates(subset='Academic Program Name')['Academic Program Name']

# Identify new branches introduced after 2019
new_branches = pd.DataFrame()

for branch in df_unique_branches:
    df_branch = df[df['Academic Program Name'] == branch]
    if df_branch['Year'].min() > 2019:
        new_branches = pd.concat([new_branches, df_branch])

# Sort by average rank
sorted_by_popularity = new_branches.sort_values(by='Average Rank')

# Select the top 10 branches based on the sorted order
top_10 = sorted_by_popularity.drop_duplicates(subset=['Academic Program Name', 'Institute']).head(10)

# Display the top 10 most popular new branches
print("Top 10 Most Popular New Branches Based on Average Rank:")
print(top_10[['Academic Program Name', 'Institute']])

# Create a Plotly bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=top_10['Average Rank'],
    y=top_10['Institute-Academic Program Name'],
    orientation='h',
    marker=dict(color=top_10['Average Rank'], coloraxis="coloraxis")
))

fig.update_layout(
    title='Average Rank of Top 10 Different Branches',
    xaxis_title='Average Rank',
    yaxis_title='Academic Program Name - Institute',
    yaxis=dict(autorange='reversed'),
    coloraxis=dict(colorscale='Blues'),
    height=800
)
fig.show()

# If the data includes multiple years, perform trend analysis
if 'Year' in df.columns:
    # Filter the data for the trend analysis
    trend_data = df_filtered[df_filtered['Academic Program Name'].isin(top_10['Academic Program Name'])]
    
    # Plot trends for the selected branches
    for branch in top_10['Institute-Academic Program Name']:
        branch_data = trend_data[trend_data['Institute-Academic Program Name'] == branch]
        fig.add_trace(go.Scatter(
            x=branch_data['Year'],
            y=branch_data['Average Rank'],
            mode='lines+markers',
            name=f'{branch} - Average Rank'
        ))

    fig.update_layout(
        title='Trend of Average Ranks Over Years for Top New Branches',
        xaxis_title='Year',
        yaxis_title='Average Rank',
        legend_title='Branches'
    )
fig.update_xaxes(range=[2021, 2023]) 
fig.show()

# JavaScript for hover functionality
hover_js = """
<script>
    var myPlot = document.getElementsByClassName('plotly-graph-div')[0];

    myPlot.on('plotly_hover', function(data){
        var index = data.points[0].curveNumber;
        Plotly.restyle(myPlot, 'opacity', Array.from({length: data.points.length}, (_, i) => i === index ? 1 : 0.2));
    });

    myPlot.on('plotly_unhover', function(data){
        Plotly.restyle(myPlot, 'opacity', 0.2);
    });
</script>
"""

display(HTML(hover_js))


C:\Users\91934\AppData\Local\Temp\ipykernel_30452\3399847211.py:7: DtypeWarning:

Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.



Top 10 Most Popular New Branches Based on Average Rank:
                                    Academic Program Name  \
365764  Mathematics and Computing (4 Years, Bachelor o...   
335663  Mathematics & Computing (5 Years, Bachelor of ...   
314069  Artificial Intelligence and Data Science (4 Ye...   
307727  Computer Science and Engineering with Speciali...   
271713  Integrated B. Tech.(IT) and M. Tech (IT) (5 Ye...   
217035  Engineering and Computational Mechanics (4 Yea...   
321862  Computer Science Engineering (Artificial lntel...   
151536  Computer Science and Artificial Intelligence (...   
321725  Artificial Intelligence and Data Science (4 Ye...   
313548  Artificial Intelligence and Data Science (4 Ye...   

                                                Institute  
365764          Indian Institute  of Technology Kharagpur  
335663            Indian Institute  of Technology Roorkee  
314069              Indian Institute  of Technology Patna  
307727             National Inst

In [33]:
##7 Preparatory Rank List
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('combined2.csv')

# Ensure the relevant columns are present and convert ranks to numeric
df['Opening Rank'] = pd.to_numeric(df['Opening Rank'], errors='coerce')
df['Closing Rank'] = pd.to_numeric(df['Closing Rank'], errors='coerce')
df.dropna(subset=['Institute','Opening Rank'],inplace=True)

# Drop rows with NaN values in 'Opening Rank' or 'Closing Rank'
df_prep = df[df['Closing Rank'].isnull()]

seatty = input("Enter Your seat type(OPEN,EWS,OBC-NCL,SC,ST,EWS (PwD),OBC-NCL (PwD),OPEN (PwD), SC (PwD), ST(PwD)):")
gender = input("Enter Your Gender(Male,Female):")
year = int(input('Enter year you want to check'))

seatty = seatty.upper()
gender = gender.lower()
gender_list = ['Female-only (including Supernumerary)'] if gender == 'female' else ['Gender-Neutral']

prep_list = df_prep[(df['Round']==6) &(df_prep['Seat Type'] == seatty) & (df_prep['Gender'].isin(gender_list)) ]
prep_list.sort_values(by = 'Opening Rank',ascending = False)
print(prep_list[['Institute','Academic Program Name','Opening Rank']])


C:\Users\91934\AppData\Local\Temp\ipykernel_22672\2832582472.py:6: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined2.csv')


ValueError: invalid literal for int() with base 10: ''

In [53]:
##8 corresponding rounds prediction
import numpy as np
import pandas as pd

df = pd.read_csv('combined2.csv')
df['Opening Rank'] = pd.to_numeric(df['Opening Rank'], errors='coerce')
df['Closing Rank'] = pd.to_numeric(df['Closing Rank'], errors='coerce')
df.dropna(subset = 'Closing Rank',inplace=True)

seatty = input("Enter Your seat type(OPEN,EWS,OBC-NCL,SC,ST,EWS (PwD),OBC-NCL (PwD),OPEN (PwD), SC (PwD), ST(PwD)):")
gender = input("Enter Your Gender(Male,Female):")
quota = input('Enter Your Quota(AI,HS,OS):')

seatty = seatty.upper()
gender = gender.lower()
quota = quota.upper()
gender_list = ['Female-only (including Supernumerary)'] if gender == 'female' else ['Gender-Neutral']

df = df[(df['Year']==2023) & (df['Seat Type'] == seatty) & (df['Gender'].isin(gender_list)) & (df['Quota'] == quota)]
target_branch = input('Enter Target Branch:')
target_college = input('Enter Target College:')
rank = int(input('Enter Rank'))
rank_target = int(input('Enter Target branch Current Closing Rank'))
round_no = int(input('Enter Current Round Number'))
df_refined = df[(df['Academic Program Name'].str.contains(target_branch, case=False)) & (df['Institute'] == target_college) & ((df['Round'] == round_no) | (df['Round'] == 6))]
branches = df_refined.drop_duplicates(subset='Academic Program Name', keep='first')
for branch in branches['Academic Program Name']:
    df_temp = df_refined[(df_refined['Round'] == round_no)]
    df_6 = df_refined[(df_refined['Round'] == 6)]
    diff = df_temp['Closing Rank'].values[0] - df_6['Closing Rank'].values[0]
    diff = diff*0.9
    if (diff > (rank - rank_target) & (rank <= df_6['Closing Rank'].values[0]) ) :
        print(f'Can get {branch} in {target_college} in final round')
    else :
        print(f'Cannot get {branch} in {target_college} in final round')

C:\Users\91934\AppData\Local\Temp\ipykernel_22672\3431767227.py:5: DtypeWarning: Columns (6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined2.csv')


Cannot get Mechanical Engineering (4 Years, Bachelor of Technology) in Indian Institute  of Technology Guwahati in final round
